### Profitability Exploration
I have a model right now trained on the 2021-2023 seasons however I still need to see how it performs, and if I can find an edge to earn some profits. First, I want to check how this model does against the Vegas Spread Data. Using the 2024 season as a test set, my model reached a MAE of 10.37

In [110]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [111]:
raw_data = pd.read_csv('../data/raw/spread_data/spreadspoke_scores.csv')

I'm going to check to see what the error for the spreads are on average after week 7.

In [112]:
target_segment = raw_data[raw_data['schedule_season']==2024]
target_segment['team_home'] = target_segment['team_home'].replace('Washington Football Team', 'Washington Commanders')
target_segment['team_away'] = target_segment['team_away'].replace('Washington Football Team', 'Washington Commanders')
target_segment['team_home'] = target_segment['team_home'].str.split().str[-1]

/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_18236/2346947454.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_segment['team_home'] = target_segment['team_home'].replace('Washington Football Team', 'Washington Commanders')
/var/folders/bs/x8kwr7tx2yn1x05tvj7m0nc40000gn/T/ipykernel_18236/2346947454.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  target_segment['team_away'] = target_segment['team_away'].replace('Washington Football Team', 'Washington Commanders')
/var/folders/bs/x8

In [113]:
target_segment[target_segment['team_home'] == 'Commanders']

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
13831,9/15/2024,2024,2,False,Commanders,21.0,18.0,New York Giants,WAS,-1.5,42.5,FedEx Field,False,NaN,NaN,NaN,NaN
13877,10/6/2024,2024,5,False,Commanders,34.0,13.0,Cleveland Browns,WAS,-3.0,44,FedEx Field,False,NaN,NaN,NaN,NaN
13905,10/20/2024,2024,7,False,Commanders,40.0,7.0,Carolina Panthers,WAS,-10.0,51,FedEx Field,False,NaN,NaN,NaN,NaN
13922,10/27/2024,2024,8,False,Commanders,18.0,15.0,Chicago Bears,WAS,-1.5,46.5,FedEx Field,False,NaN,NaN,NaN,NaN
13951,11/10/2024,2024,10,False,Commanders,27.0,28.0,Pittsburgh Steelers,WAS,-2.0,45,FedEx Field,False,NaN,NaN,NaN,NaN
13978,11/24/2024,2024,12,False,Commanders,26.0,34.0,Dallas Cowboys,WAS,-10.5,44.5,FedEx Field,False,NaN,NaN,NaN,NaN
13994,12/1/2024,2024,13,False,Commanders,42.0,19.0,Tennessee Titans,WAS,-6.0,45,FedEx Field,False,41.0,6.0,40.0,NaN
14039,12/22/2024,2024,16,False,Commanders,36.0,33.0,Philadelphia Eagles,PHI,-4.0,47,FedEx Field,False,NaN,NaN,NaN,NaN
14055,12/29/2024,2024,17,False,Commanders,30.0,24.0,Atlanta Falcons,WAS,-3.5,46.5,FedEx Field,False,NaN,NaN,NaN,NaN


In [114]:
nfl_abbrev_map = {
    'ARI': 'Cardinals',
    'ATL': 'Falcons',
    'BAL': 'Ravens',
    'BUF': 'Bills',
    'CAR': 'Panthers',
    'CHI': 'Bears',
    'CIN': 'Bengals',
    'CLE': 'Browns',
    'DAL': 'Cowboys',
    'DEN': 'Broncos',
    'DET': 'Lions',
    'GB':  'Packers',
    'HOU': 'Texans',
    'IND': 'Colts',
    'JAX': 'Jaguars',
    'KC':  'Chiefs',
    'LAC': 'Chargers',
    'LAR': 'Rams',
    'LVR': 'Raiders',
    'MIA': 'Dolphins',
    'MIN': 'Vikings',
    'NE':  'Patriots',
    'NO':  'Saints',
    'NYG': 'Giants',
    'NYJ': 'Jets',
    'PHI': 'Eagles',
    'PIT': 'Steelers',
    'SEA': 'Seahawks',
    'SF':  '49ers',
    'TB':  'Buccaneers',
    'TEN': 'Titans',
    'WAS': 'Commanders',
}

In [115]:
# Create a copy to be safe
df_odds = target_segment.copy()

# 1. Map the abbreviations to Mascot names
df_odds['fav_team'] = df_odds['team_favorite_id'].map(nfl_abbrev_map)

# 2. Check for missing values (CRITICAL STEP)
# If an abbreviation wasn't in the dictionary, it will turn into NaN.
missing_teams = df_odds[df_odds['fav_team'].isna()]['team_favorite_id'].unique()

if len(missing_teams) > 0:
    print(f"Warning! The following abbreviations were not found in the dictionary: {missing_teams}")
else:
    print("Success! All favorites mapped to mascots.")

df_odds.head()

Success! All favorites mapped to mascots.


,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail,fav_team
13801,9/5/2024,2024,1,False,Chiefs,27.0,20.0,Baltimore Ravens,KC,-3.0,47,GEHA Field at Arrowhead Stadium,False,NaN,NaN,NaN,NaN,Chiefs
13802,9/6/2024,2024,1,False,Eagles,34.0,29.0,Green Bay Packers,PHI,-2.0,49.5,Corinthians Arena,True,NaN,NaN,NaN,NaN,Eagles
13803,9/8/2024,2024,1,False,Falcons,10.0,18.0,Pittsburgh Steelers,ATL,-3.5,42,Mercedes-Benz Stadium,False,72.0,0.0,NaN,indoor,Falcons
13804,9/8/2024,2024,1,False,Bills,34.0,28.0,Arizona Cardinals,BUF,-6.5,46.5,Highmark Stadium,False,NaN,NaN,NaN,NaN,Bills
13805,9/8/2024,2024,1,False,Bears,24.0,17.0,Tennessee Titans,CHI,-4.0,44.5,Soldier Field,False,NaN,NaN,NaN,NaN,Bears


In [116]:
df_odds['acc_spread'] = np.where(
    df_odds['fav_team'] == df_odds['team_home'], 
    df_odds['score_away'] - df_odds['score_home'],      # True: Home is Fav
    df_odds['score_home'] - df_odds['score_away']       # False: Away is Fav
)

In [117]:
df_odds['Abs_Error'] = np.abs(df_odds['acc_spread'] - df_odds['spread_favorite'])

In [118]:
def filter_weeks(x):
    try:
        # Try converting to a number and checking if > 7
        return float(x) > 7
    except ValueError:
        # If conversion fails (it's a real string like "Playoffs"), keep it
        return True

midseason = df_odds[df_odds['schedule_week'].apply(filter_weeks)]
midseason['Abs_Error'].mean()

np.float64(9.258426966292134)

Unfortunately, the Vegas lines beat out my model with a MAE of 9.26 (my model has a 10.37 MAE) which isn't super surprising given Vegas lines are known to be some of the most accurate models. This means if I want to make any profit, I have to explore other directions.

### Strategic Picking

While over the coures of 2/3 of the season, my model on average was about 1.11 points less accurate than Vegas lines which isn't terrible but can not outright beat them averaged across every game. However, there are games where my model does beat out Vegas' prediction. I have to isolate those data points where my model outperforms the set line, and be able to predict when and which games these are in the future.